In [1]:
import os
import shutil
from tqdm import tqdm


base_path = '/kaggle/input/sounddata' 


npy_destination = '/kaggle/working/collected_preprocessed_train'
label_destination = '/kaggle/working/collected_labels'

os.makedirs(npy_destination, exist_ok=True)
os.makedirs(label_destination, exist_ok=True)

npy_files_count = 0
label_files_count = 0

# Loop through each directory (001 to 006)
for i in range(1, 7):
    folder_name = f'HF_Lung_V1-20240819T223709Z-00{i}'
    preprocessed_train_path = os.path.join(base_path, folder_name, 'HF_Lung_V1/preprocessed_train')
    train_path = os.path.join(base_path, folder_name, 'HF_Lung_V1/train')
    
    # Collecting all .npy files from preprocessed_train
    if os.path.exists(preprocessed_train_path):
        for npy_file in tqdm(os.listdir(preprocessed_train_path), desc=f"Copying .npy files from {folder_name}"):
            if npy_file.endswith('.npy'):
                npy_source = os.path.join(preprocessed_train_path, npy_file)
                shutil.copy(npy_source, npy_destination)
                npy_files_count += 1
    
    # Collecting all label files from train
    if os.path.exists(train_path):
        for label_file in tqdm(os.listdir(train_path), desc=f"Copying label files from {folder_name}"):
            if label_file.endswith('_label.txt'):
                label_source = os.path.join(train_path, label_file)
                shutil.copy(label_source, label_destination)
                label_files_count += 1

npy_destination_files = len(os.listdir(npy_destination))
label_destination_files = len(os.listdir(label_destination))

print(f"Expected .npy files: {npy_files_count}, Copied .npy files: {npy_destination_files}")
print(f"Expected label files: {label_files_count}, Copied label files: {label_destination_files}")

if npy_files_count == npy_destination_files:
    print("All .npy files have been successfully copied.")
else:
    print("Warning: Some .npy files might be missing.")

if label_files_count == label_destination_files:
    print("All label files have been successfully copied.")
else:
    print("Warning: Some label files might be missing.")

Copying .npy files from HF_Lung_V1-20240819T223709Z-001: 100%|██████████| 899/899 [00:18<00:00, 48.54it/s]
Copying label files from HF_Lung_V1-20240819T223709Z-001: 100%|██████████| 3323/3323 [00:12<00:00, 273.19it/s]
Copying .npy files from HF_Lung_V1-20240819T223709Z-002: 100%|██████████| 1594/1594 [00:33<00:00, 47.29it/s]
Copying label files from HF_Lung_V1-20240819T223709Z-002: 100%|██████████| 3584/3584 [00:17<00:00, 208.41it/s]
Copying .npy files from HF_Lung_V1-20240819T223709Z-003: 100%|██████████| 1671/1671 [00:41<00:00, 40.27it/s]
Copying label files from HF_Lung_V1-20240819T223709Z-003: 100%|██████████| 2110/2110 [00:10<00:00, 198.92it/s]
Copying .npy files from HF_Lung_V1-20240819T223709Z-004: 100%|██████████| 1619/1619 [00:36<00:00, 43.90it/s]
Copying label files from HF_Lung_V1-20240819T223709Z-004: 100%|██████████| 131/131 [00:00<00:00, 309.35it/s]
Copying .npy files from HF_Lung_V1-20240819T223709Z-005: 100%|██████████| 1564/1564 [00:33<00:00, 47.18it/s]
Copying label f

Expected .npy files: 7809, Copied .npy files: 7809
Expected label files: 7809, Copied label files: 7809
All .npy files have been successfully copied.
All label files have been successfully copied.


In [2]:
import os

# Paths to the collected files
npy_destination = '/kaggle/working/collected_preprocessed_train'
label_destination = '/kaggle/working/collected_labels'

# Get lists of files in each directory
npy_files = sorted([f for f in os.listdir(npy_destination) if f.endswith('.npy')])
label_files = sorted([f for f in os.listdir(label_destination) if f.endswith('_label.txt')])

# Extract the base filenames (without extensions) for comparison
npy_ids = set([os.path.splitext(f)[0].replace('_processed', '') for f in npy_files])
label_ids = set([os.path.splitext(f)[0].replace('_label', '') for f in label_files])

# Find missing matches
missing_labels = npy_ids - label_ids
missing_npy = label_ids - npy_ids

# Output results
if not missing_labels and not missing_npy:
    print("All .npy files have corresponding label files.")
else:
    if missing_labels:
        print(f"Missing labels for the following .npy files: {missing_labels}")
    if missing_npy:
        print(f"Missing .npy files for the following labels: {missing_npy}")

All .npy files have corresponding label files.


### BiLSTM & BiGRU

In [3]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score

# Function to convert time string (hh:mm:ss.ms) to an index
def time_to_index(time_str, total_length, duration):
    h, m, s = map(float, time_str.split(':'))
    time_in_seconds = h * 3600 + m * 60 + s
    index = int((time_in_seconds / duration) * total_length)
    return index

# Function to parse labels and convert them into binary vectors
def parse_label_file(label_file_path, sequence_length, duration):
    label_vector = np.zeros(sequence_length)
    with open(label_file_path, 'r') as file:
        label_content = file.readlines()
    for line in label_content:
        match = re.match(r'(\w) (\d{2}:\d{2}:\d{2}\.\d{3}) (\d{2}:\d{2}:\d{2}\.\d{3})', line)
        if match:
            event, start_time, end_time = match.groups()
            start_index = time_to_index(start_time, sequence_length, duration)
            end_index = time_to_index(end_time, sequence_length, duration)
            # Marking the segment for the event as 1 (presence of event)
            label_vector[start_index:end_index] = 1
    return label_vector

# Custom dataset class for lung sound data
class LungSoundDataset(Dataset):
    def __init__(self, npy_files, label_files, sequence_length=938, duration=15):
        self.features = np.array([np.load(f) for f in npy_files])  # Convert list of arrays to a single array
        self.labels = np.array([parse_label_file(f, sequence_length, duration) for f in label_files])

        # Ensure the correct shape for the input (batch_size, sequence_length, feature_size)
        self.features = torch.FloatTensor(self.features[:, :sequence_length, :]).transpose(1, 2)  # Transpose to [batch_size, sequence_length, feature_size]
        self.labels = torch.FloatTensor(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Bidirectional LSTM/GRU model definition
class BiRecurrentModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, rnn_type='BiLSTM'):
        super(BiRecurrentModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        if rnn_type == 'BiLSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        elif rnn_type == 'BiGRU':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        else:
            raise ValueError("rnn_type must be 'BiLSTM' or 'BiGRU'")
        
        self.fc1 = nn.Linear(hidden_size * 2, 32)  # hidden_size * 2 because of bidirection
        self.fc2 = nn.Linear(32, 1)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        if isinstance(self.rnn, nn.LSTM):
            c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
            out, _ = self.rnn(x, (h0, c0))
        else:
            out, _ = self.rnn(x, h0)
        
        out = self.fc1(out)  
        out = self.fc2(out)
        out = torch.sigmoid(out)  
        return out.squeeze(-1) 

def check_and_fix_nan(tensor, name, default_value=0.0):
    if torch.isnan(tensor).any():
        print(f"NaN detected in {name}, replacing with {default_value}")
        tensor = torch.where(torch.isnan(tensor), torch.tensor(default_value).to(tensor.device), tensor)
    return tensor

def check_and_fix_bounds(tensor, name, min_val=0.0, max_val=1.0):
    if (tensor < min_val).any() or (tensor > max_val).any():
        print(f"Out-of-bounds detected in {name}, clamping to [{min_val}, {max_val}]")
        tensor = torch.clamp(tensor, min=min_val, max=max_val)
        tensor = torch.where(tensor < min_val, torch.tensor(min_val).to(tensor.device), tensor)
        tensor = torch.where(tensor > max_val, torch.tensor(max_val).to(tensor.device), tensor)
    return tensor

# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, fold, model_type):
    model.to(device)
    best_val_auc = 0.0
    best_model_path = None
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for features, labels in tqdm(train_loader, desc=f"{model_type} Epoch {epoch+1}/{num_epochs}"):
            features, labels = features.to(device), labels.to(device)
            
            features = check_and_fix_nan(features, "features")
            features = check_and_fix_bounds(features, "features")
            
            labels = check_and_fix_nan(labels, "labels")
            labels = check_and_fix_bounds(labels, "labels")

            optimizer.zero_grad()
            outputs = model(features)

            outputs = check_and_fix_nan(outputs, "raw model outputs")
            outputs = check_and_fix_bounds(outputs, "raw model outputs")

            outputs = torch.sigmoid(outputs)

            outputs = check_and_fix_nan(outputs, "sigmoid outputs")
            outputs = check_and_fix_bounds(outputs, "sigmoid outputs")

            loss = criterion(outputs, labels)

            loss = check_and_fix_nan(loss, "loss")

            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            for name, param in model.named_parameters():
                if param.grad is not None:
                    param.grad = check_and_fix_nan(param.grad, f"gradient of {name}")

            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation step
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_outputs = []
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)

                features = check_and_fix_nan(features, "validation features")
                features = check_and_fix_bounds(features, "validation features")

                labels = check_and_fix_nan(labels, "validation labels")
                labels = check_and_fix_bounds(labels, "validation labels")
                
                outputs = model(features)

                outputs = check_and_fix_nan(outputs, "validation raw outputs")
                outputs = check_and_fix_bounds(outputs, "validation raw outputs")

                outputs = torch.sigmoid(outputs)
                outputs = check_and_fix_bounds(outputs, "validation clamped outputs")
                
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                all_labels.append(labels.cpu().numpy())
                all_outputs.append(outputs.cpu().numpy())
        
        all_labels = np.concatenate(all_labels).flatten()
        all_outputs = np.concatenate(all_outputs).flatten()
        
        val_auc = roc_auc_score(all_labels, all_outputs)
        val_f1 = f1_score(all_labels, all_outputs > 0.5)
        
        print(f"{model_type} Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"Val AUC: {val_auc:.4f}, Val F1: {val_f1:.4f}")
        
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_model_path = f"/kaggle/working/best_{model_type}_model_fold_{fold+1}_epoch_{epoch+1}.pth"
            torch.save(model.state_dict(), best_model_path)

    return best_model_path

# Cross-validation setup and training
def run_cross_validation(npy_files, label_files, input_size, hidden_size, num_layers, num_epochs, learning_rate, device):
    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    best_models = {'BiLSTM': [], 'BiGRU': []}
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(npy_files)):
        print(f"Training fold {fold+1}/{kfold.n_splits}")
        
        train_npy_files = [npy_files[i] for i in train_idx]
        val_npy_files = [npy_files[i] for i in val_idx]
        train_label_files = [label_files[i] for i in train_idx]
        val_label_files = [label_files[i] for i in val_idx]
        
        train_dataset = LungSoundDataset(train_npy_files, train_label_files)
        val_dataset = LungSoundDataset(val_npy_files, val_label_files)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_dataset, batch_size=32, num_workers=4)
        
        # Training BiLSTM model
        bilstm_model = BiRecurrentModel(input_size, hidden_size, num_layers, 'BiLSTM')
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(bilstm_model.parameters(), lr=learning_rate)
        best_bilstm_model_path = train_model(bilstm_model, train_loader, val_loader, criterion, optimizer, num_epochs, device, fold, 'BiLSTM')
        best_models['BiLSTM'].append(best_bilstm_model_path)
        
        # Training BiGRU model
        bigru_model = BiRecurrentModel(input_size, hidden_size, num_layers, 'BiGRU')
        optimizer = torch.optim.Adam(bigru_model.parameters(), lr=learning_rate)
        best_bigru_model_path = train_model(bigru_model, train_loader, val_loader, criterion, optimizer, num_epochs, device, fold, 'BiGRU')
        best_models['BiGRU'].append(best_bigru_model_path)
    
    print(f"Best BiLSTM models from each fold: {best_models['BiLSTM']}")
    print(f"Best BiGRU models from each fold: {best_models['BiGRU']}")
    return best_models

# Paths to the preprocessed features and label files
npy_destination = '/kaggle/working/collected_preprocessed_train'
label_destination = '/kaggle/working/collected_labels'

npy_files = sorted([os.path.join(npy_destination, f) for f in os.listdir(npy_destination) if f.endswith('.npy')])
label_files = sorted([os.path.join(label_destination, f) for f in os.listdir(label_destination) if f.endswith('_label.txt')])

# Parameters for model training
input_size = 193  # Feature dimension
hidden_size = 128  # Hidden units in RNN cells
num_layers = 2  # Number of RNN layers
num_epochs = 5  # Number of epochs to train
learning_rate = 0.0001  # Learning rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run cross-validation and train models
best_model_paths = run_cross_validation(npy_files, label_files, input_size, hidden_size, num_layers, num_epochs, learning_rate, device)
print("Training complete. Best models saved from each fold:", best_model_paths)


Training fold 1/3


BiLSTM Epoch 1/5:  21%|██        | 34/163 [01:17<04:49,  2.24s/it]

NaN detected in features, replacing with 0.0


BiLSTM Epoch 1/5: 100%|██████████| 163/163 [06:23<00:00,  2.35s/it]


NaN detected in validation features, replacing with 0.0
BiLSTM Epoch 1/5
Train Loss: 0.7073
Val Loss: 0.6933
Val AUC: 0.3965, Val F1: 0.5114


...

NaN detected in features, replacing with 0.0


BiGRU Epoch 5/5:  88%|████████▊ | 144/163 [06:40<00:52,  2.75s/it]

NaN detected in features, replacing with 0.0


BiGRU Epoch 5/5: 100%|██████████| 163/163 [07:32<00:00,  2.78s/it]


BiGRU Epoch 5/5
Train Loss: 0.6932
Val Loss: 0.6932
Val AUC: 0.4108, Val F1: 0.5130
Best BiLSTM models from each fold: ['/kaggle/working/best_BiLSTM_model_fold_1_epoch_1.pth', '/kaggle/working/best_BiLSTM_model_fold_2_epoch_1.pth', '/kaggle/working/best_BiLSTM_model_fold_3_epoch_2.pth']
Best BiGRU models from each fold: ['/kaggle/working/best_BiGRU_model_fold_1_epoch_5.pth', '/kaggle/working/best_BiGRU_model_fold_2_epoch_1.pth', '/kaggle/working/best_BiGRU_model_fold_3_epoch_1.pth']
Training complete. Best models saved from each fold: {'BiLSTM': ['/kaggle/working/best_BiLSTM_model_fold_1_epoch_1.pth', '/kaggle/working/best_BiLSTM_model_fold_2_epoch_1.pth', '/kaggle/working/best_BiLSTM_model_fold_3_epoch_2.pth'], 'BiGRU': ['/kaggle/working/best_BiGRU_model_fold_1_epoch_5.pth', '/kaggle/working/best_BiGRU_model_fold_2_epoch_1.pth', '/kaggle/working/best_BiGRU_model_fold_3_epoch_1.pth']}


### CNN added

In [1]:
import os
import re
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as nn_utils
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score

# Function to convert time string (hh:mm:ss.ms) to an index
def time_to_index(time_str, total_length, duration):
    h, m, s = map(float, time_str.split(':'))
    time_in_seconds = h * 3600 + m * 60 + s
    index = int((time_in_seconds / duration) * total_length)
    return index

# Function to parse labels and convert them into binary vectors
def parse_label_file(label_file_path, sequence_length, duration):
    label_vector = np.zeros(sequence_length)
    with open(label_file_path, 'r') as file:
        label_content = file.readlines()
    for line in label_content:
        match = re.match(r'(\w) (\d{2}:\d{2}:\d{2}\.\d{3}) (\d{2}:\d{2}:\d{2}\.\d{3})', line)
        if match:
            event, start_time, end_time = match.groups()
            start_index = time_to_index(start_time, sequence_length, duration)
            end_index = time_to_index(end_time, sequence_length, duration)
            # Marking the segment for the event as 1 (presence of event)
            label_vector[start_index:end_index] = 1
    return label_vector

# Custom dataset class for lung sound data
class LungSoundDataset(Dataset):
    def __init__(self, npy_files, label_files, sequence_length=938, duration=15):
        self.features = np.array([np.load(f) for f in npy_files])  # Convert list of arrays to a single array
        self.labels = np.array([parse_label_file(f, sequence_length, duration) for f in label_files])

        # Ensure the correct shape for the input (batch_size, sequence_length, feature_size)
        self.features = torch.FloatTensor(self.features[:, :sequence_length, :]).transpose(1, 2)  # Transpose to [batch_size, sequence_length, feature_size]
        self.labels = torch.FloatTensor(self.labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# CNN + RNN model definition
class CNNRecurrentModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, rnn_type='LSTM', bidirectional=False):
        super(CNNRecurrentModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.output_size = hidden_size * 2 if bidirectional else hidden_size

        # CNN layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(6, 6), padding=(2, 2))  # (N, 938, 193, 1) -> (N, 938, 193, 64)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(4, 4), padding=(1, 1))  # (N, 938, 193, 64) -> (N, 469, 97, 64)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout_cnn = nn.Dropout(0.3)

        # Calculate the flattened size after CNN layers
        self.flattened_size = self.calculate_flattened_size(input_size)

        # RNN layers
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size=self.flattened_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size=self.flattened_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        else:
            raise ValueError("rnn_type must be 'LSTM' or 'GRU'")

        # Fully connected layers
        self.fc1 = nn.Linear(self.output_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout_rnn = nn.Dropout(0.3)

    def calculate_flattened_size(self, input_size):
        dummy_input = torch.zeros(1, 1, 938, input_size)
        x = self.conv1(dummy_input)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.bn2(x)
        x = F.relu(x)
        flattened_size = x.view(x.size(0), x.size(2), -1).size(-1)
        return flattened_size

    def forward(self, x):
        # CNN forward pass
        x = x.unsqueeze(1)  # Add a channel dimension (N, 1, 938, 193)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout_cnn(x)
        x = x.view(x.size(0), x.size(2), -1)  # Flatten the output

        # RNN forward pass
        h0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
        if isinstance(self.rnn, nn.LSTM):
            c0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
            out, _ = self.rnn(x, (h0, c0))
        else:
            out, _ = self.rnn(x, h0)

        out = self.dropout_rnn(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)
        return out.squeeze(-1)

# Function to check and fix NaN values
def check_and_fix_nan(tensor, name, default_value=0.0):
    if torch.isnan(tensor).any():
        print(f"NaN detected in {name}, replacing with {default_value}")
        tensor = torch.where(torch.isnan(tensor), torch.tensor(default_value).to(tensor.device), tensor)
    return tensor

# Function to check and fix bounds
def check_and_fix_bounds(tensor, name, min_val=0.0, max_val=1.0):
    if (tensor < min_val).any() or (tensor > max_val).any():
        print(f"Out-of-bounds detected in {name}, clamping to [{min_val}, {max_val}]")
        tensor = torch.clamp(tensor, min=min_val, max=max_val)
        # Ensure strict bounds by forcefully setting values outside [0,1]
        tensor = torch.where(tensor < min_val, torch.tensor(min_val).to(tensor.device), tensor)
        tensor = torch.where(tensor > max_val, torch.tensor(max_val).to(tensor.device), tensor)
    return tensor

# Custom loss function to handle different sizes of outputs and labels
def custom_loss(outputs, labels):
    if outputs.size(1) != labels.size(1):
        labels = F.interpolate(labels.unsqueeze(1), size=outputs.size(1), mode='linear', align_corners=False).squeeze(1)
    loss = nn.BCELoss()(outputs, labels)
    return loss

# Function to train the model
def train_model(model, train_loader, val_loader, optimizer, num_epochs, device, model_type):
    model.to(device)
    best_val_auc = 0.0
    best_model_path = None
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for features, labels in tqdm(train_loader, desc=f"{model_type} Epoch {epoch+1}/{num_epochs}"):
            features, labels = features.to(device), labels.to(device)
            
            # Check and fix NaN and bounds in features and labels
            features = check_and_fix_nan(features, "features")
            features = check_and_fix_bounds(features, "features")
            
            labels = check_and_fix_nan(labels, "labels")
            labels = check_and_fix_bounds(labels, "labels")

            optimizer.zero_grad()
            outputs = model(features)

            # Check and fix NaN and bounds in raw model outputs
            outputs = check_and_fix_nan(outputs, "raw model outputs")
            outputs = check_and_fix_bounds(outputs, "raw model outputs")

            # Apply sigmoid activation to ensure outputs are between 0 and 1
            outputs = torch.sigmoid(outputs)

            # Check and fix NaN and bounds in sigmoid outputs
            outputs = check_and_fix_nan(outputs, "sigmoid outputs")
            outputs = check_and_fix_bounds(outputs, "sigmoid outputs")

            loss = custom_loss(outputs, labels)

            # Check and fix NaN in loss
            loss = check_and_fix_nan(loss, "loss")

            loss.backward()
            
            # Apply gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Check and fix NaN in gradients
            for name, param in model.named_parameters():
                if param.grad is not None:
                    param.grad = check_and_fix_nan(param.grad, f"gradient of {name}")

            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation step (similar NaN and bounds checks as above)
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_outputs = []
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)

                # Check and fix NaN and bounds in features and labels during validation
                features = check_and_fix_nan(features, "validation features")
                features = check_and_fix_bounds(features, "validation features")

                labels = check_and_fix_nan(labels, "validation labels")
                labels = check_and_fix_bounds(labels, "validation labels")
                
                outputs = model(features)

                # Check and fix NaN and bounds in raw outputs during validation
                outputs = check_and_fix_nan(outputs, "validation raw outputs")
                outputs = check_and_fix_bounds(outputs, "validation raw outputs")

                outputs = torch.sigmoid(outputs)
                outputs = check_and_fix_bounds(outputs, "validation clamped outputs")
                
                loss = custom_loss(outputs, labels)
                val_loss += loss.item()
                
                all_labels.append(labels.cpu().numpy())
                all_outputs.append(outputs.cpu().numpy())
        
        all_labels = np.concatenate(all_labels).flatten()
        all_outputs = np.concatenate(all_outputs).flatten()
        
        val_auc = roc_auc_score(all_labels, all_outputs)
        val_f1 = f1_score(all_labels, all_outputs > 0.5)
        
        print(f"{model_type} Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"Val AUC: {val_auc:.4f}, Val F1: {val_f1:.4f}")
        
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_model_path = f"/kaggle/working/best_{model_type}_Model_epoch_{epoch+1}.pth"
            torch.save(model.state_dict(), best_model_path)

    return best_model_path

# Paths to the preprocessed features and label files
npy_destination = '/kaggle/working/collected_preprocessed_train'
label_destination = '/kaggle/working/collected_labels'

npy_files = sorted([os.path.join(npy_destination, f) for f in os.listdir(npy_destination) if f.endswith('.npy')])
label_files = sorted([os.path.join(label_destination, f) for f in os.listdir(label_destination) if f.endswith('_label.txt')])

# Parameters for model training
input_size = 193  # Adjusted input size after CNN flattening
hidden_size = 128  # Hidden units in RNN cells
num_layers = 2  # Number of RNN layers
num_epochs = 2  # Number of epochs to train
learning_rate = 0.0001  # Learning rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare datasets and loaders
train_dataset = LungSoundDataset(npy_files, label_files)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(train_dataset, batch_size=32, num_workers=4)  # Using train set as validation set for simplicity

# Train CNN-LSTM model
cnn_lstm_model = CNNRecurrentModel(input_size, hidden_size, num_layers, 'LSTM', bidirectional=False)
optimizer = torch.optim.Adam(cnn_lstm_model.parameters(), lr=learning_rate)
best_cnn_lstm_model_path = train_model(cnn_lstm_model, train_loader, val_loader, optimizer, num_epochs, device, 'CNN_LSTM')

# Train CNN-GRU model
cnn_gru_model = CNNRecurrentModel(input_size, hidden_size, num_layers, 'GRU', bidirectional=False)
optimizer = torch.optim.Adam(cnn_gru_model.parameters(), lr=learning_rate)
best_cnn_gru_model_path = train_model(cnn_gru_model, train_loader, val_loader, optimizer, num_epochs, device, 'CNN_GRU')

# Train CNN-BiLSTM model
cnn_bilstm_model = CNNRecurrentModel(input_size, hidden_size, num_layers, 'LSTM', bidirectional=True)
optimizer = torch.optim.Adam(cnn_bilstm_model.parameters(), lr=learning_rate)
best_cnn_bilstm_model_path = train_model(cnn_bilstm_model, train_loader, val_loader, optimizer, num_epochs, device, 'CNN_BiLSTM')

# Train CNN-BiGRU model
cnn_bigru_model = CNNRecurrentModel(input_size, hidden_size, num_layers, 'GRU', bidirectional=True)
optimizer = torch.optim.Adam(cnn_bigru_model.parameters(), lr=learning_rate)
best_cnn_bigru_model_path = train_model(cnn_bigru_model, train_loader, val_loader, optimizer, num_epochs, device, 'CNN_BiGRU')

print(f"Best CNN-LSTM model path: {best_cnn_lstm_model_path}")
print(f"Best CNN-GRU model path: {best_cnn_gru_model_path}")
print(f"Best CNN-BiLSTM model path: {best_cnn_bilstm_model_path}")
print(f"Best CNN-BiGRU model path: {best_cnn_bigru_model_path}")


CNN_LSTM Epoch 1/2:   2%|▏         | 4/245 [02:58<2:59:44, 44.75s/it]

KeyboardInterrupt

